In [55]:
import os
import pandas as pd
import zipfile
import sqlite3

### __Download kaggle.com datasets__
---  

<b>NFL Draft dataset</b>  
<b>Download:</b> Download NFL draft data from Kaggle using the following statement in terminal <em>-d ulrikthygepedersen/nfl-draft-1985-2015</em>  

<b>NFL Combine dataset</b>  
<b>Download:</b> Download NFL combine data Kaggle using the following statement in terminal <em>-d mitchellweg1/nfl-combine-results-dataset-2000-2022</em>


### __Read datasets into dataframes__
---

In [56]:

# Extract NFL draft zip file
cur_folder = 'C:\\Bootcamp_Git\\project_4_healthcare\\NFL_data'
input_path = os.path.join(cur_folder, 'nfl-draft-1985-2015.zip')
with zipfile.ZipFile(input_path, 'r') as zip_ref:
    names = zip_ref.namelist()
    zip_ref.extractall(cur_folder)

#  Read into pandas dataframe and delete zip extract
zip_extract = os.path.join(cur_folder, names[0])
nfl_df = pd.read_csv(zip_extract)
os.remove(zip_extract)

# ---Step 3: extract combine data---
input_path = os.path.join(cur_folder, 'nfl-combine-results-dataset-2000-2022.zip')
with zipfile.ZipFile(input_path, 'r') as zip_ref:
    names = zip_ref.namelist()
    zip_ref.extractall(cur_folder)

# Read into pandas dataframe and delete zip extract
combine_df = pd.DataFrame()
for name in names:
    zip_extract = os.path.join(cur_folder, name)
    combine_files_df = pd.read_csv(zip_extract)
    combine_files_df['year'] = name[:4]
    combine_df = pd.concat([combine_df, combine_files_df], ignore_index=True)
    os.remove(zip_extract)


### __Remove columns__
---  
<b>NFL Draft:</b>  
<ol>
<li><b>Removed:</b> column_a, player_id, and year because they are unique to this dataset and will not be used in model.</li>  
<li><b>Removed:</b> the following columns because they refer to NFL stats after the player was drafted.  
tm, hof, position_standard, first4av, age, to, ap1, pb, st, carav, drav, g, cmp, pass_att, pass_yds, pass_td, pass_int, rush_att, rush_yds, rush_tds, rec, rec_yds, rec_tds, tkl, def_int, and sk.</li></ol>  
  

<b>NFL Combine:</b>  
<ol>
<li><b>Removed:</b> All fields will be used.</li></ol>

In [57]:
# Remove NFL draft columns
nfl_df = nfl_df[['player', 'year', 'pos', 'rnd', 'pick', 'college_univ']]
nfl_df.head()

,player,year,pos,rnd,pick,college_univ
0,Jameis Winston,2015,QB,1,1,Florida St.
1,Marcus Mariota,2015,QB,1,2,Oregon
2,Dante Fowler,2015,OLB,1,3,Florida
3,Amari Cooper,2015,WR,1,4,Alabama
4,Brandon Scherff,2015,T,1,5,Iowa


### __Exploratory Analysis__
---

In [58]:
# Shape of dataframes
print(f'NFL draft dataframe shape: {nfl_df.shape}')
print(f'NFL combine dataframe shape: {combine_df.shape}')

NFL draft dataframe shape: (8435, 6)
NFL combine dataframe shape: (7680, 12)


In [59]:
# Number of columns by data type
print(f'NFL draft dataframe columns by type:\n{nfl_df.dtypes.value_counts()}\n')
print(f'NFL combine dataframe columns by type:\n{combine_df.dtypes.value_counts()}')

NFL draft dataframe columns by type:
object    3
int64     3
Name: count, dtype: int64

NFL combine dataframe columns by type:
float64    7
object     5
Name: count, dtype: int64


In [60]:
# Check for null values
print(f'NFL draft dataframe null values:\n')
nfl_df.info()

NFL draft dataframe null values:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8435 entries, 0 to 8434
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   player        8435 non-null   object
 1   year          8435 non-null   int64 
 2   pos           8435 non-null   object
 3   rnd           8435 non-null   int64 
 4   pick          8435 non-null   int64 
 5   college_univ  5005 non-null   object
dtypes: int64(3), object(3)
memory usage: 395.5+ KB


In [61]:
# Check for null values
print(f'NFL combine dataframe null values:\n')
combine_df.info()

NFL combine dataframe null values:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7680 entries, 0 to 7679
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Player      7680 non-null   object 
 1   Pos         7680 non-null   object 
 2   School      7680 non-null   object 
 3   Ht          7651 non-null   object 
 4   Wt          7656 non-null   float64
 5   40yd        7206 non-null   float64
 6   Vertical    5932 non-null   float64
 7   Bench       5096 non-null   float64
 8   Broad Jump  5859 non-null   float64
 9   3Cone       4792 non-null   float64
 10  Shuttle     4895 non-null   float64
 11  year        7680 non-null   object 
dtypes: float64(7), object(5)
memory usage: 720.1+ KB


In [62]:
# Remove players with NaN college_univ in NFL draft dataframe
nfl_df = nfl_df.dropna(subset=['college_univ'])

In [63]:
# Create player id column to merge dataframes
# Add player_id column to dataframe year + 4 character last name + 4 character first name because of same name for different players
nfl_df['player_id'] = nfl_df['year'].astype(str) + nfl_df['player'].str.split(' ').str[1].str.upper() + nfl_df['player'].str.split(' ').str[0].str.upper() + nfl_df['college_univ'].str[:3].str.upper()
combine_df['player_id'] = combine_df['year'].astype(str) + combine_df['Player'].str.split(' ').str[1].str.upper() + combine_df['Player'].str.split(' ').str[0].str.upper()  + combine_df['School'].str[:3].str.upper()

In [64]:
# Check for duplicate player ids
print(f'NFL draft dataframe duplicate player ids: {nfl_df["player_id"].duplicated().sum()}')
print(f'NFL combine dataframe duplicate player ids: {combine_df["player_id"].duplicated().sum()}')

NFL draft dataframe duplicate player ids: 1
NFL combine dataframe duplicate player ids: 2


In [65]:
# View duplicate player ids
print(f'NFL draft dataframe duplicate player ids:\n')
nfl_df[nfl_df["player_id"].duplicated()]

NFL draft dataframe duplicate player ids:



,player,year,pos,rnd,pick,college_univ,player_id
968,Robert Griffin,2012,OL,6,203,Baylor,2012GRIFFINROBERTBAY


In [66]:
# View duplicate player ids
print(f'NFL combine dataframe duplicate player ids:\n')
combine_df[combine_df["player_id"].duplicated()]

NFL combine dataframe duplicate player ids:



,Player,Pos,School,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,year,player_id
1784,Derrick Johnson,OLB,Texas,6-3,242.0,4.52,37.5,NaN,120.0,7.20,3.87,2005,2005JOHNSONDERRICKTEX
2352,Buster Davis,ILB,Florida State,5-9,239.0,4.64,31.0,24.0,104.0,7.28,4.37,2007,2007DAVISBUSTERFLO


In [67]:
# drop records where player_id = '2012GRIFFINROBERTBAY' and pos = 'OL'
nfl_drop_index = nfl_df.loc[(nfl_df['player_id'] == '2012GRIFFINROBERTBAY') & (nfl_df['pos'] == 'OL')].index
nfl_df.drop(nfl_drop_index, inplace=True)

# drop records where player_id = '2007DAVISBUSTERFLO' and pos = 'WR'
combine_drop_index = combine_df.loc[(combine_df['player_id'] == '2007DAVISBUSTERFLO') & (combine_df['Pos'] == 'WR')].index
combine_df.drop(combine_drop_index, inplace=True)

# drop records where player_id = '2005JOHNSONDERRICKTEX' and pos = 'CB'
combine_drop_index = combine_df.loc[(combine_df['player_id'] == '2005JOHNSONDERRICKTEX') & (combine_df['Pos'] == 'CB')].index
combine_df.drop(combine_drop_index, inplace=True)

In [68]:
# View duplicate player ids
print(f'NFL draft dataframe duplicate player ids:\n')
nfl_df[nfl_df["player_id"].duplicated()]

NFL draft dataframe duplicate player ids:



,player,year,pos,rnd,pick,college_univ,player_id


In [69]:
# View duplicate player ids
print(f'NFL combine dataframe duplicate player ids:\n')
combine_df[combine_df["player_id"].duplicated()]

NFL combine dataframe duplicate player ids:



,Player,Pos,School,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,year,player_id


In [70]:
# Merge dataframes on player_id
draft_df = combine_df.merge(nfl_df, on=['player_id'], how='left')

In [71]:
# Check column names
print(f'NFL draft dataframe column names:\n')
draft_df.columns

NFL draft dataframe column names:



Index(['Player', 'Pos', 'School', 'Ht', 'Wt', '40yd', 'Vertical', 'Bench',
       'Broad Jump', '3Cone', 'Shuttle', 'year_x', 'player_id', 'player',
       'year_y', 'pos', 'rnd', 'pick', 'college_univ'],
      dtype='object')

In [72]:
# Drop duplicate columns
draft_df.drop(['player', 'year_y', 'pos', 'college_univ'], axis=1, inplace=True)

In [73]:
# Rename fields
draft_df.rename(columns={'Player': 'player_name', 'Pos': 'position', 'School': 'school', 'Ht': 'height', 'year_x': 'year', 'Wt': 'weight', '40yd': 'forty_yard',
                         'Vertical': 'vertical_leap', 'Bench': 'bench_press', 'Broad Jump': 'broad_jump', 'Shuttle': 'shuttle_run',
                         '3Cone': 'three_cone', 'rnd': 'draft_round', 'pick': 'draft_pick'}, inplace=True)


In [74]:
# Add column ht for height in inches
draft_df[['ht', 'in']] = draft_df['height'].str.split('-', expand=True)
draft_df['in'].fillna(0, inplace=True)
draft_df['ht'].fillna(0, inplace=True)
draft_df['ht'] = draft_df['ht'].astype(int) * 12
draft_df['ht'] = draft_df['ht'] + draft_df['in'].astype(int)
draft_df.drop(['in'], axis=1, inplace=True)

In [75]:
# Replace '-' with ' ft ' in 'Ht' column and add ' in' to end of 'Ht' column for Tableau dashboard
draft_df['height'] = draft_df['height'].str.replace('-', ' ft ')
draft_df['height'] = draft_df['height'] + ' in'

In [76]:
# Drop players with NaN in following columns forty_yard, vertical_leap, bench_press, broad_jump, three_cone, and shuttle_run
draft_df.dropna(subset=['forty_yard', 'vertical_leap', 'bench_press', 'broad_jump', 'three_cone', 'shuttle_run'], inplace=True, how='all')

In [77]:
# Convert year to int
draft_df['year'] = draft_df['year'].astype(int)

In [78]:
# Limit to years 2000-2015
draft_df = draft_df.loc[(draft_df['year'] >= 2000) & (draft_df['year'] <= 2015)]

### __Ranking__
---  
Create score columns for combine events where if >= 3rd quartile = 4 elseif >= median elseif >=  1st quartile = 2 else = 1 and NaN = 0.  
The points will flip for running events because the lower score is better.

In [79]:
# create list of positions
pos_list = ['C', 'CB', 'DE', 'DT', 'FB', 'ILB', 'K', 'LS', 'OG', 'OLB', 'OT', 'P', 'QB', 'RB', 'S', 'TE', 'WR']

In [80]:
# add column for bench_press if >= 3rd quartile = 4 elseif >= median elseif >=  1st quartile = 2 else = 1 do not include NaN
event_combined_df = pd.DataFrame()
for pos in pos_list:
    event_df = draft_df[(draft_df['bench_press'].notna()) & (draft_df['position'] == pos)].copy()
    if len(event_df) == 0:
        pass
    else:
        event_df['bench_press_score'] = 0
        event_df.loc[event_df['bench_press'] >= event_df['bench_press'].quantile(.75), 'bench_press_score'] = 4
        event_df.loc[(event_df['bench_press'] >= event_df['bench_press'].median()) & (event_df['bench_press'] < event_df['bench_press'].quantile(.75)), 'bench_press_score'] = 3
        event_df.loc[(event_df['bench_press'] >= event_df['bench_press'].quantile(.25)) & (event_df['bench_press'] < event_df['bench_press'].median()), 'bench_press_score'] = 2
        event_df.loc[event_df['bench_press'] < event_df['bench_press'].quantile(.25), 'bench_press_score'] = 1
        event_combined_df = pd.concat([event_combined_df, event_df], ignore_index=True)


draft_df = draft_df.merge(event_combined_df[['player_id', 'bench_press_score']], on=['player_id'], how='left')

In [81]:
# add column for broad_jump if >= 3rd quartile = 4 elseif >= median elseif >=  1st quartile = 2 else = 1 do not include NaN
event_combined_df = pd.DataFrame()
for pos in pos_list:
    event_df = draft_df[(draft_df['broad_jump'].notna()) & (draft_df['position'] == pos)].copy()
    if len(event_df) == 0:
        pass
    else:
        event_df['broad_jump_score'] = 0
        event_df.loc[event_df['broad_jump'] >= event_df['broad_jump'].quantile(.75), 'broad_jump_score'] = 4
        event_df.loc[(event_df['broad_jump'] >= event_df['broad_jump'].median()) & (event_df['broad_jump'] < event_df['broad_jump'].quantile(.75)), 'broad_jump_score'] = 3
        event_df.loc[(event_df['broad_jump'] >= event_df['broad_jump'].quantile(.25)) & (event_df['broad_jump'] < event_df['broad_jump'].median()), 'broad_jump_score'] = 2
        event_df.loc[event_df['broad_jump'] < event_df['broad_jump'].quantile(.25), 'broad_jump_score'] = 1
        event_combined_df = pd.concat([event_combined_df, event_df], ignore_index=True)

draft_df = draft_df.merge(event_combined_df[['player_id', 'broad_jump_score']], on=['player_id'], how='left')


In [82]:
# add column for forty_yard if >= 3rd quartile = 4 elseif >= median elseif >=  1st quartile = 2 else = 1 do not include NaN
event_combined_df = pd.DataFrame()
for pos in pos_list:
    event_df = draft_df[(draft_df['forty_yard'].notna()) & (draft_df['position'] == pos)].copy()
    if len(event_df) == 0:
        pass
    else:
        event_df['forty_yard_score'] = 0
        event_df.loc[event_df['forty_yard'] <= event_df['forty_yard'].quantile(.25), 'forty_yard_score'] = 4
        event_df.loc[(event_df['forty_yard'] <= event_df['forty_yard'].median()) & (event_df['forty_yard'] > event_df['forty_yard'].quantile(.25)), 'forty_yard_score'] = 3
        event_df.loc[(event_df['forty_yard'] <= event_df['forty_yard'].quantile(.75)) & (event_df['forty_yard'] > event_df['forty_yard'].median()), 'forty_yard_score'] = 2
        event_df.loc[event_df['forty_yard'] > event_df['forty_yard'].quantile(.75), 'forty_yard_score'] = 1
        event_combined_df = pd.concat([event_combined_df, event_df], ignore_index=True)

draft_df = draft_df.merge(event_combined_df[['player_id', 'forty_yard_score']], on=['player_id'], how='left')


In [83]:
# add column for shuttle_run if >= 3rd quartile = 4 elseif >= median elseif >=  1st quartile = 2 else = 1 do not include NaN
event_combined_df = pd.DataFrame()
for pos in pos_list:
    event_df = draft_df[(draft_df['shuttle_run'].notna()) & (draft_df['position'] == pos)].copy()
    if len(event_df) == 0:
        pass
    else:
        event_df['shuttle_run_score'] = 0
        event_df.loc[event_df['shuttle_run'] <= event_df['shuttle_run'].quantile(.25), 'shuttle_run_score'] = 4
        event_df.loc[(event_df['shuttle_run'] <= event_df['shuttle_run'].median()) & (event_df['shuttle_run'] > event_df['shuttle_run'].quantile(.25)), 'shuttle_run_score'] = 3
        event_df.loc[(event_df['shuttle_run'] <= event_df['shuttle_run'].quantile(.75)) & (event_df['shuttle_run'] > event_df['shuttle_run'].median()), 'shuttle_run_score'] = 2
        event_df.loc[event_df['shuttle_run'] > event_df['shuttle_run'].quantile(.75), 'shuttle_run_score'] = 1
        event_combined_df = pd.concat([event_combined_df, event_df], ignore_index=True)

draft_df = draft_df.merge(event_combined_df[['player_id', 'shuttle_run_score']], on=['player_id'], how='left')


In [84]:
# add column for three_cone if >= 3rd quartile = 4 elseif >= median elseif >=  1st quartile = 2 else = 1 do not include NaN
event_combined_df = pd.DataFrame()
for pos in pos_list:
    event_df = draft_df[(draft_df['three_cone'].notna()) & (draft_df['position'] == pos)].copy()
    if len(event_df) == 0:
        pass
    else:
        event_df['three_cone_score'] = 0
        event_df.loc[event_df['three_cone'] <= event_df['three_cone'].quantile(.25), 'three_cone_score'] = 4
        event_df.loc[(event_df['three_cone'] <= event_df['three_cone'].median()) & (event_df['three_cone'] > event_df['three_cone'].quantile(.25)), 'three_cone_score'] = 3
        event_df.loc[(event_df['three_cone'] <= event_df['three_cone'].quantile(.75)) & (event_df['three_cone'] > event_df['three_cone'].median()), 'three_cone_score'] = 2
        event_df.loc[event_df['three_cone'] > event_df['three_cone'].quantile(.75), 'three_cone_score'] = 1
        event_combined_df = pd.concat([event_combined_df, event_df], ignore_index=True)

draft_df = draft_df.merge(event_combined_df[['player_id', 'three_cone_score']], on=['player_id'], how='left')


In [85]:
# add column for vertical_leap if >= 3rd quartile = 4 elseif >= median elseif >=  1st quartile = 2 else = 1 do not include NaN
event_combined_df = pd.DataFrame()
for pos in pos_list:
    event_df = draft_df[(draft_df['vertical_leap'].notna()) & (draft_df['position'] == pos)].copy()
    if len(event_df) == 0:
        pass
    else:
        event_df['vertical_leap_score'] = 0
        event_df.loc[event_df['vertical_leap'] >= event_df['vertical_leap'].quantile(.75), 'vertical_leap_score'] = 4
        event_df.loc[(event_df['vertical_leap'] >= event_df['vertical_leap'].median()) & (event_df['vertical_leap'] < event_df['vertical_leap'].quantile(.75)), 'vertical_leap_score'] = 3
        event_df.loc[(event_df['vertical_leap'] >= event_df['vertical_leap'].quantile(.25)) & (event_df['vertical_leap'] < event_df['vertical_leap'].median()), 'vertical_leap_score'] = 2
        event_df.loc[event_df['vertical_leap'] < event_df['vertical_leap'].quantile(.25), 'vertical_leap_score'] = 1
        event_combined_df = pd.concat([event_combined_df, event_df], ignore_index=True)

draft_df = draft_df.merge(event_combined_df[['player_id', 'vertical_leap_score']], on=['player_id'], how='left')

In [86]:
# fill NaN with 0 for columns bench_press_score, broad_jump_score, forty_yard_score, shuttle_run_score, three_cone_score, and vertical_leap_score
draft_df[['bench_press_score', 'broad_jump_score', 'forty_yard_score', 'shuttle_run_score', 'three_cone_score', 'vertical_leap_score']] = draft_df[['bench_press_score', 'broad_jump_score', 'forty_yard_score', 'shuttle_run_score', 'three_cone_score', 'vertical_leap_score']].fillna(0)

In [87]:
# add column for bench_press_score + broad_jump_score + forty_yard_score + shuttle_run_score + three_cone_score + vertical_leap_score
draft_df['combine_score'] = draft_df['bench_press_score'] + draft_df['broad_jump_score'] + draft_df['forty_yard_score'] + draft_df['shuttle_run_score'] + draft_df['three_cone_score'] + draft_df['vertical_leap_score']

In [88]:
# add column if draft_pick is NaN = 0 else = 1
draft_df['draft_pick_flag'] = 0
draft_df.loc[draft_df['draft_pick'].notna(), 'draft_pick_flag'] = 1

In [89]:
# update draft_round to 8 if draft_pick is NaN
draft_df.loc[draft_df['draft_pick'].isna(), 'draft_round'] = 8

In [90]:
# update draft_pick to 999 if draft_pick is NaN
draft_df.loc[draft_df['draft_pick'].isna(), 'draft_pick'] = 999

### __Fillna__
---  
Populate combine events that have NaN with the median of the column based on player's position.

In [91]:
# for column vertical_leap, bench_press, broad_jump, shuttle_run, three_cone replace NaN with the median of the column based on position_combine and draft_pick_flag
draft_df['vertical_leap'] = draft_df.groupby(['position', 'draft_pick_flag'])['vertical_leap'].transform(lambda x: x.fillna(x.median()))
draft_df['bench_press'] = draft_df.groupby(['position', 'draft_pick_flag'])['bench_press'].transform(lambda x: x.fillna(x.median()))
draft_df['broad_jump'] = draft_df.groupby(['position', 'draft_pick_flag'])['broad_jump'].transform(lambda x: x.fillna(x.median()))
draft_df['shuttle_run'] = draft_df.groupby(['position', 'draft_pick_flag'])['shuttle_run'].transform(lambda x: x.fillna(x.median()))
draft_df['three_cone'] = draft_df.groupby(['position', 'draft_pick_flag'])['three_cone'].transform(lambda x: x.fillna(x.median()))
draft_df['forty_yard'] = draft_df.groupby(['position', 'draft_pick_flag'])['forty_yard'].transform(lambda x: x.fillna(x.median()))

In [92]:
# # for column vertical_leap, broad_jump, shuttle_run and three_cone replace NaN with the median of the column because kicker and punter did not have scores
# # to populate based on position_combine
draft_df['shuttle_run'] = draft_df['shuttle_run'].fillna(draft_df['shuttle_run'].median())
draft_df['three_cone'] = draft_df['three_cone'].fillna(draft_df['three_cone'].median())
draft_df['vertical_leap'] = draft_df['vertical_leap'].fillna(draft_df['vertical_leap'].median())
draft_df['broad_jump'] = draft_df['broad_jump'].fillna(draft_df['broad_jump'].median())

In [93]:
# Reorder columns because Tableau is not recognizing the change in column position
draft_df = draft_df[['player_id', 'player_name', 'year', 'school', 'position', 'height', 'ht', 'weight', 'forty_yard', 'vertical_leap', 'bench_press',
                     'broad_jump', 'three_cone', 'shuttle_run', 'draft_round', 'draft_pick', 'bench_press_score', 'broad_jump_score', 'forty_yard_score',
                     'shuttle_run_score', 'three_cone_score', 'vertical_leap_score', 'combine_score', 'draft_pick_flag']]

In [94]:
# Check for null values
print(f'draft dataframe null values:\n')
draft_df.info()

draft dataframe null values:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5183 entries, 0 to 5182
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   player_id            5183 non-null   object 
 1   player_name          5183 non-null   object 
 2   year                 5183 non-null   int32  
 3   school               5183 non-null   object 
 4   position             5183 non-null   object 
 5   height               5183 non-null   object 
 6   ht                   5183 non-null   int32  
 7   weight               5183 non-null   float64
 8   forty_yard           5183 non-null   float64
 9   vertical_leap        5183 non-null   float64
 10  bench_press          5183 non-null   float64
 11  broad_jump           5183 non-null   float64
 12  three_cone           5183 non-null   float64
 13  shuttle_run          5183 non-null   float64
 14  draft_round          5183 non-null   float64
 15  draft_pi

### __Database__
---  
Create SQLite database and write draft_df to players table.

In [95]:

# write df to sqlite
conn = sqlite3.connect('C:/Project_Git/project_4_fantasy_football/Resources/database.db')
draft_df.to_sql('players', conn, if_exists='replace', index=False)
conn.close()

In [96]:
# confirm data was written to sqlite
conn = sqlite3.connect('C:/Project_Git/project_4_fantasy_football/Resources/database.db')
cursor = conn.cursor()
cursor.execute("SELECT count(*) FROM players")
print(cursor.fetchall())
conn.close()

[(5183,)]


In [97]:
# write draft_df to csv
draft_df.to_csv('C:/Project_Git/project_4_fantasy_football/Resources/tableau_data.csv', index=False)

### __RAW data__
---  
The team requested I add a table to database with raw data. All columns no transformation.

In [98]:

# Extract NFL draft zip file
cur_folder = 'C:\\Bootcamp_Git\\project_4_healthcare\\NFL_data'
input_path = os.path.join(cur_folder, 'nfl-draft-1985-2015.zip')
with zipfile.ZipFile(input_path, 'r') as zip_ref:
    names = zip_ref.namelist()
    zip_ref.extractall(cur_folder)

#  Read into pandas dataframe and delete zip extract
zip_extract = os.path.join(cur_folder, names[0])
nfl_df = pd.read_csv(zip_extract)
os.remove(zip_extract)

# ---Step 3: extract combine data---
input_path = os.path.join(cur_folder, 'nfl-combine-results-dataset-2000-2022.zip')
with zipfile.ZipFile(input_path, 'r') as zip_ref:
    names = zip_ref.namelist()
    zip_ref.extractall(cur_folder)

# Read into pandas dataframe and delete zip extract
combine_df = pd.DataFrame()
for name in names:
    zip_extract = os.path.join(cur_folder, name)
    combine_files_df = pd.read_csv(zip_extract)
    combine_files_df['year'] = name[:4]
    combine_df = pd.concat([combine_df, combine_files_df], ignore_index=True)
    os.remove(zip_extract)

In [99]:
# Create player id column to merge dataframes
# Add player_id column to dataframe year + 4 character last name + 4 character first name because of same name for different players
nfl_df['player_id'] = nfl_df['year'].astype(str) + nfl_df['player'].str.split(' ').str[1].str.upper() + nfl_df['player'].str.split(' ').str[0].str.upper() + nfl_df['college_univ'].str[:3].str.upper()
combine_df['player_id'] = combine_df['year'].astype(str) + combine_df['Player'].str.split(' ').str[1].str.upper() + combine_df['Player'].str.split(' ').str[0].str.upper()  + combine_df['School'].str[:3].str.upper()

In [100]:
# drop records where player_id = '2012GRIFFINROBERTBAY' and pos = 'OL'
nfl_drop_index = nfl_df.loc[(nfl_df['player_id'] == '2012GRIFFINROBERTBAY') & (nfl_df['pos'] == 'OL')].index
nfl_df.drop(nfl_drop_index, inplace=True)

# drop records where player_id = '2007DAVISBUSTERFLO' and pos = 'WR'
combine_drop_index = combine_df.loc[(combine_df['player_id'] == '2007DAVISBUSTERFLO') & (combine_df['Pos'] == 'WR')].index
combine_df.drop(combine_drop_index, inplace=True)

# drop records where player_id = '2005JOHNSONDERRICKTEX' and pos = 'CB'
combine_drop_index = combine_df.loc[(combine_df['player_id'] == '2005JOHNSONDERRICKTEX') & (combine_df['Pos'] == 'CB')].index
combine_df.drop(combine_drop_index, inplace=True)

In [101]:
# Merge dataframes on player_id
draft_raw_df = combine_df.merge(nfl_df, on=['player_id'], how='left')

In [102]:
# Drop duplicate columns
draft_raw_df.drop(['player', 'year_y', 'pos', 'college_univ'], axis=1, inplace=True)

In [103]:
# rename year_x to year
draft_raw_df.rename(columns={'Player': 'player_name', 'Pos': 'position', 'School': 'school', 'Ht': 'height', 'year_x': 'year', 'Wt': 'weight', '40yd': 'forty_yard',
                         'Vertical': 'vertical_leap', 'Bench': 'bench_press', 'Broad Jump': 'broad_jump', 'Shuttle': 'shuttle_run',
                         '3Cone': 'three_cone', 'rnd': 'draft_round', 'pick': 'draft_pick'}, inplace=True)

In [110]:
# Drop players with NaN in following columns forty_yard, vertical_leap, bench_press, broad_jump, three_cone, and shuttle_run
draft_raw_df.dropna(subset=['forty_yard', 'vertical_leap', 'bench_press', 'broad_jump', 'three_cone', 'shuttle_run'], inplace=True, how='all')

In [111]:
# Convert year to int
draft_raw_df['year'] = draft_raw_df['year'].astype(int)

In [112]:
# Limit to years 2000-2015
draft_raw_df = draft_raw_df.loc[(draft_raw_df['year'] >= 2000) & (draft_raw_df['year'] <= 2015)]

In [113]:
# write draft_df to csv
draft_raw_df.to_csv('C:/Project_Git/project_4_fantasy_football/Resources/tableau_raw_data.csv', index=False)

In [108]:
# write df to sqlite
conn = sqlite3.connect('C:/Project_Git/project_4_fantasy_football/Resources/database.db')
draft_raw_df.to_sql('players_raw', conn, if_exists='replace', index=False)
conn.close()

In [109]:
# confirm data was written to sqlite
conn = sqlite3.connect('C:/Project_Git/project_4_fantasy_football/Resources/database.db')
cursor = conn.cursor()
cursor.execute("SELECT count(*) FROM players_raw")
print(cursor.fetchall())
conn.close()

[(5183,)]
